## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPY_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luanda,AO,-8.8368,13.2343,77.00,88,75,10.36,broken clouds
1,1,Albany,US,42.6001,-73.9662,48.38,94,80,2.35,broken clouds
2,2,Kenai,US,60.5544,-151.2583,24.15,96,90,3.44,light snow
3,3,Assiniboia,CA,49.6334,-105.9845,-16.69,63,65,13.80,broken clouds
4,4,Vaini,TO,-21.2000,-175.2000,77.16,100,90,13.80,moderate rain


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luanda,AO,-8.8368,13.2343,77.00,88,75,10.36,broken clouds
4,4,Vaini,TO,-21.2000,-175.2000,77.16,100,90,13.80,moderate rain
5,5,Praia,CV,14.9215,-23.5087,75.74,60,0,13.80,clear sky
7,7,Hilo,US,19.7297,-155.0900,77.61,93,90,3.44,light rain
10,10,Puerto Baquerizo Moreno,EC,-0.9000,-89.6000,76.19,78,75,13.80,broken clouds
16,16,Cabedelo,BR,-6.9811,-34.8339,83.07,71,52,12.08,broken clouds
17,17,Alofi,NU,-19.0595,-169.9187,80.49,83,100,8.05,light rain
19,19,Rikitea,PF,-23.1203,-134.9692,77.76,73,18,12.10,light rain
20,20,Georgetown,MY,5.4112,100.3354,80.53,85,20,5.99,few clouds
23,23,Kapaa,US,22.0752,-159.3190,77.47,85,75,1.99,light rain


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                246
City                   246
Country                245
Lat                    246
Lng                    246
Max Temp               246
Humidity               246
Cloudiness             246
Wind Speed             246
Current Description    246
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Luanda,AO,77.00,broken clouds,-8.8368,13.2343,
4,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,
5,Praia,CV,75.74,clear sky,14.9215,-23.5087,
7,Hilo,US,77.61,light rain,19.7297,-155.0900,
10,Puerto Baquerizo Moreno,EC,76.19,broken clouds,-0.9000,-89.6000,
16,Cabedelo,BR,83.07,broken clouds,-6.9811,-34.8339,
17,Alofi,NU,80.49,light rain,-19.0595,-169.9187,
19,Rikitea,PF,77.76,light rain,-23.1203,-134.9692,
20,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
23,Kapaa,US,77.47,light rain,22.0752,-159.3190,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df["Hotel Name"].dropna()

In [33]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Luanda,AO,77.00,broken clouds,-8.8368,13.2343,Hotel Fórum
4,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
5,Praia,CV,75.74,clear sky,14.9215,-23.5087,LT Aparthotel
7,Hilo,US,77.61,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Puerto Baquerizo Moreno,EC,76.19,broken clouds,-0.9000,-89.6000,Hotel los Algarrobos
16,Cabedelo,BR,83.07,broken clouds,-6.9811,-34.8339,Hotel Pigalle
17,Alofi,NU,80.49,light rain,-19.0595,-169.9187,Taloa Heights
19,Rikitea,PF,77.76,light rain,-23.1203,-134.9692,People ThankYou
20,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
23,Kapaa,US,77.47,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} at {Max Temp}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))